In [2]:
import json
import os
import random
import time

In [3]:
import cloudscraper
import geopandas as gpd
import pandas as pd
import h3

In [4]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [5]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [6]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [7]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [8]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [9]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Burger King Data

In [10]:
scraper = cloudscraper.create_scraper()

In [11]:
url = "https://use1-prod-bk-gateway.rbictg.com/graphql"
headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7",
    "apollographql-client-name": "wl-rn-web",
    "apollographql-client-version": "7.48.0-7.48.0-no-uid-80341dd",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-forter-token": "e18e49d691f449b58ac38163f1fbfb81_1743979237613__UDF43-m4_13ck_",
    "x-platform-framework": "react-dom",
    "x-session-id": "1743979237610",
    "x-ui-language": "en",
    "x-ui-platform": "web",
    "x-ui-region": "US",
    "x-ui-version": "7.48.0",
    "x-user-datetime": "2025-05-06T14:59:34-07:00",
}

In [14]:
national_restaurant_list = []
res = 4
for i in tqdm(range(len(states_df)), desc="States"):
    state_restaurant_list = []

    state_code = states_df.iloc[i]["STUSPS"]

    # Skip territories
    if state_code in ["PR", "VI", "GU", "MP", "AS"]:
        continue

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    state_shape = states_df.iloc[i]["geometry"]
    h3_shape = h3.geo_to_h3shape(state_shape.buffer(distance=0.2))
    h3_cells = h3.h3shape_to_cells(h3_shape, res=res)
    for h3_cell in tqdm(
        h3_cells, desc=f"Parsing state: {state_code} in Resolution: {res}"
    ):
        lat, lng = h3.cell_to_latlng(h3_cell)
        params = {
            "operationName": "GetNearbyRestaurants",
            "variables": json.dumps(
                {
                    "input": {
                        "pagination": {"first": 100},
                        "radiusStrictMode": False,
                        "coordinates": {
                            "searchRadius": 32000,
                            "userLat": lat,
                            "userLng": lng,
                        },
                    }
                }
            ),
            "extensions": json.dumps(
                {
                    "persistedQuery": {
                        "version": 1,
                        "sha256Hash": "823984086ffe25adca8294186fc551ab2aa4830da4e851f91298d423adab5b20",
                    }
                }
            ),
        }
        r = scraper.get(url=url, params=params, headers=headers)
        store_jsons = r.json()["data"]["restaurantsV2"]["nearby"]["nodes"]
        for store_json in store_jsons:
            store_id = store_json["storeId"]
            store_state_code = store_json["physicalAddress"]["stateProvinceShort"]
            store_lat = store_json["latitude"]
            store_long = store_json["longitude"]

            if state_code != store_state_code:
                continue
            else:
                store_dict = {
                    "ID": store_id,
                    "STUSPS": store_state_code,
                    "geometry": Point(store_long, store_lat),
                }
                state_restaurant_list.append(store_dict)

    state_bk_gdf = gpd.GeoDataFrame(state_restaurant_list, crs=4326)
    state_bk_gdf = state_bk_gdf.drop_duplicates("ID")
    state_bk_gdf = state_bk_gdf[state_bk_gdf["STUSPS"] == state_code]
    state_bk_gdf.to_file(f"data/states/{state_code}.gpkg")

States:   0%|          | 0/56 [00:00<?, ?it/s]

Parsing state: IL in Resolution: 4:   0%|          | 0/106 [00:00<?, ?it/s]

Parsing state: VT in Resolution: 4:   0%|          | 0/22 [00:00<?, ?it/s]

Parsing state: MT in Resolution: 4:   0%|          | 0/258 [00:00<?, ?it/s]

Parsing state: IA in Resolution: 4:   0%|          | 0/100 [00:00<?, ?it/s]

...

Parsing state: ND in Resolution: 4:   0%|          | 0/123 [00:00<?, ?it/s]

Parsing state: AR in Resolution: 4:   0%|          | 0/99 [00:00<?, ?it/s]

Parsing state: IN in Resolution: 4:   0%|          | 0/70 [00:00<?, ?it/s]

Parsing state: MN in Resolution: 4:   0%|          | 0/147 [00:00<?, ?it/s]

Parsing state: CT in Resolution: 4:   0%|          | 0/13 [00:00<?, ?it/s]

In [15]:
bk_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        bk_gdf = gpd.read_file(constructed_path)
        bk_gdfs.append(bk_gdf)

In [16]:
bk_gdf = gpd.GeoDataFrame(pd.concat(bk_gdfs, ignore_index=True))
bk_gdf = bk_gdf.to_crs(9311)
bk_gdf.to_file(f"data/stores.gpkg")

In [23]:
bk_counts_df = (
    pd.DataFrame(bk_gdf.groupby("STUSPS").size())
    .reset_index()
    .rename(columns={0: "BKs"})
)

## Combine With States

In [28]:
bk_counts_gdf = states_with_population_df.merge(bk_counts_df, on="STUSPS", how="left")

In [29]:
bk_counts_gdf = bk_counts_gdf.fillna(0)
bk_counts_gdf["BKs"] = bk_counts_gdf["BKs"].astype(int)

In [30]:
bk_counts_gdf["per_100k"] = bk_counts_gdf["BKs"] / (
    bk_counts_gdf["POPULATION"] / 100_000
)
bk_counts_gdf["per_1m"] = bk_counts_gdf["BKs"] / (
    bk_counts_gdf["POPULATION"] / 1_000_000
)

In [31]:
bk_counts_gdf = bk_counts_gdf.fillna(0)

In [32]:
bk_counts_gdf = bk_counts_gdf[
    ["POPULATION", "geometry", "NAME", "per_100k", "per_1m", "BKs"]
]

In [33]:
bk_counts_gdf = bk_counts_gdf.to_crs(9311)
bk_counts_gdf.to_file(f"data/bks_per_state.gpkg")